### Sentence Unique Code

This notebook is used for give unique code to each sentences.

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
import re

In [ ]:
df_sent = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result.csv")
df_sent = df_sent.head(500)
df_sent

In [ ]:
# # 1-Way
# count_index = 0
# code_list = []
# for i in df_sent.sentence:
#     g = len(re.findall("\w+", i))
#     l = len(i)
#     f = df_sent.iloc[count_index].frequency
#     unique_code = f"G{g}-L{l}-F{f}-{count_index}"
#     code_list.append(unique_code)
#     count_index += 1
# df_sent["code"] = pd.DataFrame(code_list)
# df_sent

In [ ]:
# 2-Way
# df_empty = pd.DataFrame({'col_name' : []})
df_sent["code"] = pd.Series(dtype="string")
count_index = 0
for i in df_sent.sentence:
    g = len(re.findall("\w+", i))
    l = len(i)
    f = df_sent.iloc[count_index].frequency
    unique_code = f"G{g}-L{l}-F{f}-{count_index}"
    df_sent.at[count_index, "code"] = unique_code
    count_index += 1
df_sent

In [ ]:
df_sent.iloc[33]

In [ ]:
df_sent.to_csv("Clean_Spark_Sentence_Merge_Result_Code.csv", index=False)

#### Multi Processing Unique Code

In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re

In [2]:
help(multiprocessing)

Help on package multiprocessing:

NAME
    multiprocessing

MODULE REFERENCE
    https://docs.python.org/3.9/library/multiprocessing
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    # Package analogous to 'threading.py' but using processes
    #
    # multiprocessing/__init__.py
    #
    # This package is intended to duplicate the functionality (and much of
    # the API) of threading.py but uses processes instead of threads.  A
    # subpackage 'multiprocessing.dummy' has the same API but is a simple
    # wrapper for 'threading'.
    #
    # Copyright (c) 2006-2008, R Oudkerk
    # Licensed to PSF under a Contributor Agreement.
    #

PACKAGE CONTENTS
    connection
    context
    dummy (packag

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [ ]:
file_name = "Sentence_Merge.csv"
file_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Pyspark/Result/Sentence"

In [16]:
df_sent = pd.read_csv(f"{file_path}/{file_name}")
df_sent = df_sent.head(10000)
df_sent

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
9995,sen değil misin,599
9996,i̇htiyacım var,599
9997,neden burada,599
9998,peki ne istiyorsun,599


In [17]:
index_sent = zip(df_sent.index, df_sent.iloc[:,0])
zip_list = list(index_sent) # index numarası ve sentence içeriyor
#zip_list

In [6]:
# index, sentence = zip(*index_sent)

In [58]:
sent_code_list = []

In [63]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
sent_code_list = manager.list()


def sentence_code(index_num, sent):
    g = len(re.findall("\w+", sent))
    l = len(sent)
    f = df_sent.iloc[index_num].frequency
    unique_code = f"G{g}-L{l}-F{f}-{index_num}"
    sent_code_list.append([sent, f, unique_code])
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.starmap(sentence_code, zip_list) # zip_list fonksiyon için çoklu eleman gerekli olduğu için kullanıldı.
# Not: starmap çoklu iterable değerlere ihtiyaç duyan fonksiyonlar için kullanılıyor ve bu değerler zip veya itertools şeklinde olmalı. O yüzden burada pool.map yerine pool.starmap kullanıldı. 

In [67]:
df_code = pd.DataFrame(list(sent_code_list), columns=["sentence", "frequency", "code"])
df_code.sort_values(by="frequency", ascending=False, inplace=True)
df_code

,sentence,frequency,code
0,evet,1948596,G1-L4-F1948596-0
1,hayır,1250401,G1-L5-F1250401-1
2,tamam,882921,G1-L5-F882921-2
3,ne,753105,G1-L2-F753105-3
4,teşekkürler,343527,G1-L11-F343527-4
...,...,...,...
9995,şapka,647,G1-L5-F647-9258
9996,onu buraya getirin,647,G3-L18-F647-9259
9997,toparlan,647,G1-L8-F647-9260
9998,üzgün mü,647,G2-L8-F647-9261


In [ ]:
df_code.to_csv("Sentence_Merge_Code.csv", index=False)